In [2]:
#!pip install PyPDF2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 4.8 MB/s eta 0:00:00


In [3]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
import re
import os
import PyPDF2
import csv


In [4]:
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('stopwords')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [5]:
def read_pdf(file_path):
    reader = PyPDF2.PdfReader(file_path)
    texts = []
    for page in reader.pages:
      text = page.extract_text()
      text = text.strip('')
      text = text.replace('\n', ' ')
      text = re.sub(r'\s+', ' ', text)
      texts.append(text)
    texts = ''.join(texts)
    return texts

In [6]:
# import all pdfs in folder
current_dir = '/content/drive/MyDrive/ISEM2006 Group Project/src'
pdf_list = []
for file in os.listdir(current_dir):
    if file.endswith('.pdf'):
        pdf_list.append(current_dir + "/" + file)

letter_list = []
for pdf in pdf_list:
    text = read_pdf(pdf)
    letter_list.append(text)

FileNotFoundError: [Errno 2] No such file or directory: '/content/drive/MyDrive/ISEM2006 Group Project/src'

In [ ]:
filename = '/content/drive/MyDrive/ISEM2006 Group Project/' + 'output.csv'
# Open the file in write mode ('w') with newline='' to handle line endings correctly
with open(filename, 'w', newline='') as csvfile:
    # Create a CSV writer object
    csv_writer = csv.writer(csvfile)

    # Write the data
    for letter in letter_list:
        csv_writer.writerow([letter])

print(f"Data successfully exported to {filename}")


In [ ]:
import pandas as pd
text_csv = pd.read_csv('/content/drive/MyDrive/ISEM2006 Group Project/output.csv', header=None)
data = pd.DataFrame()
data['letter'] = text_csv.astype(str).apply(lambda x: ' '.join(x), axis=1)


In [ ]:
# Word tokenizer
text = data['letter'].tolist()
text = ''.join(text)
tokens = word_tokenize(text)
# Remove punctuation from each token using regex
cleaned_tokens = [re.sub(r'[^a-zA-Z0-9]', '', token) for token in tokens]
cleaned_tokens = [token for token in cleaned_tokens if token]  # Remove empty tokens
cleaned_tokens
# Remove stopwords
english_stopwords = stopwords.words('english')
english_stopwords = set(english_stopwords)

tokens_filtered = []
for w in cleaned_tokens:
  if w not in english_stopwords:
    tokens_filtered.append(w)
# Lemmatization/Stemming
wl = WordNetLemmatizer()
ps = PorterStemmer()
tokens_lemmatized = [wl.lemmatize(t) for t in tokens_filtered]
tokens_stemmed = [ps.stem(t) for t in tokens_lemmatized]
tokens = tokens_stemmed

In [ ]:
pd.DataFrame(tokens).value_counts()